In [1]:
import json
import copy
import argparse
from src.model.api import LLM_API
from src.model.llm import LLM_Model
from src.role_person_generation import generate_persona_description
from src.role_occupation_generation import generate_persona_occupation_description
import re
import json
from datetime import datetime
from collections import Counter

API_MODEL = ["gpt-3.5-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini"]

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
#1.基本选择：选择模型，数据集，模型配置，训练方式
parser.add_argument("--dataset", type=str, default="mfq30",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--model", type=str, default="gpt-3.5-turbo",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--role", type=str, default="place",
                    help="Path of the preprocessed dataset.")
parser.add_argument("--run_results", type=str, default=None,
                    help="Path of the preprocessed dataset.")
parser.add_argument("--gpu", type=int, default=3,
                help="Random seed.")
parser.add_argument("--role_num", type=int, default=2,
                help="Random seed.")
batch_size = 10
args = parser.parse_args(args=[])

with open("src/config.json", 'r') as file:
    args.config = json.load(file)

/home/gwy/miniconda3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from exp1_roleplay import init_dataset
prompt_data = init_dataset(args.dataset)

In [3]:
prompt_data[0]

{'question': "When you decide whether something is right or wrong, to what extent is the following consideration relevant to your thinking? \n'Whether or not someone suffered emotionally''",
 'question_idx': 1,
 'option': ['Not at all relevant',
  'Not very relevant',
  'Slightly relevant',
  'Somewhat relevant',
  'Very relevant',
  'Extremely relevant'],
 'user_prompt': "Question: When you decide whether something is right or wrong, to what extent is the following consideration relevant to your thinking? \n'Whether or not someone suffered emotionally'' A. Not at all relevant B. Not very relevant C. Slightly relevant D. Somewhat relevant E. Very relevant F. Extremely relevant? Answer: "}

In [4]:
args.model = "/data1/gwy/training_model/Value_trans/llama2_7b_chat_v0"
if args.model in API_MODEL:
    model = LLM_API(args.model, api_key=args.config['openai_api_key'], base_url=args.config['base_url'])
else:
    model = LLM_Model(args.model, args.gpu)
help_model =  LLM_API("gpt-3.5-turbo", api_key=args.config['openai_api_key'], base_url=args.config['base_url'])

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  7.29it/s]


In [5]:
if args.role=="base":
    personas = generate_persona_description(args.role_num)
elif args.role=="place":
    personas = generate_persona_occupation_description(args.role_num)

In [6]:
batch_size = 5
for i, inst in enumerate(prompt_data):
    answer_list = []
    for idx in range(0, len(personas), batch_size):
        temp_persons = personas[idx:idx + batch_size]
        prompt_list = []
        for per in temp_persons:
            prompt ="USER: "
            prompt += "Let's role-play. I will ask you a question and you must give me an answer. I want you to act as the person described below. Think from the person's perspective. \n\n"
            prompt += f"{per['description']} \n\n"
            prompt += "Use the given information to answer the question below. \n\n"
            prompt += inst['user_prompt']
            prompt_list.append(prompt)
            
        model_response = model.batch_respond(prompt_list)
        break
    break

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


USER: Let's role-play. I will ask you a question and you must give me an answer. I want you to act as the person described below. Think from the person's perspective. 

I am 39 years old, and my gender is female. I currently work as a cybersecurity expert in ('Jordan',), with an annual income of approximately 132451.My political inclination is globalism.My marital status is married, and I have children.I belong to South Asian and practice no denomination. As a extroverted person, I have my own views and attitudes in life.My pet ownership situation is none.Regarding social media, I regularly actively prefer to information gathering type.About my health, my current physical health status is in good health, and my mental state is having mental health issues.My family education background has a profound influence on me. My parents were often busy with work, neglecting emotional needs, with children primarily relying on grandparents for care. There have been several key turning points in my

In [7]:
model_response

["USER: Let's role-play. I will ask you a question and you must give me an answer. I want you to act as the person described below. Think from the person's perspective. \n\nI am 39 years old, and my gender is female. I currently work as a cybersecurity expert in ('Jordan',), with an annual income of approximately 132451.My political inclination is globalism.My marital status is married, and I have children.I belong to South Asian and practice no denomination. As a extroverted person, I have my own views and attitudes in life.My pet ownership situation is none.Regarding social media, I regularly actively prefer to information gathering type.About my health, my current physical health status is in good health, and my mental state is having mental health issues.My family education background has a profound influence on me. My parents were often busy with work, neglecting emotional needs, with children primarily relying on grandparents for care. There have been several key turning points i